In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.4: Fast Mistral patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

#Add LoRA adapters
We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


#Data Prepration
Load Data


In [ ]:
import pandas as pd

In [ ]:
!pip install huggingface_hub

In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Manish123hi/traindataset", split = "train")

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/48.2M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9912 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/495 [00:00<?, ? examples/s]

In [ ]:
PROMPT = """
In this task, you are required to evaluate an IELTS Writing Task 2 essay. Consider the following four criteria and provide a detailed assessment for each, along with a suggested band score:

## Task Achievement:
- Evaluate how well the candidate has addressed the given task.
- Assess the clarity and coherence of the response in presenting ideas.
- Identify if the candidate has fully covered all parts of the task and supported arguments appropriately.
- Suggested Band Score (Task Achievement): [Insert Score]

## Coherence and Cohesion:
- Assess the overall organization and structure of the essay.
- Evaluate the use of linking devices to connect ideas and paragraphs.
- Identify if there is a logical flow of information.
- Suggested Band Score (Coherence and Cohesion): [Insert Score]

## Lexical Resource (Vocabulary):
- Examine the range and accuracy of vocabulary used in the essay.
- Point out specific mistakes in vocabulary, such as inaccuracies or overuse of certain words and Suggest modified versions or alternatives for the identified mistakes. [list of mistakes and rectify]
- Assess the appropriateness of vocabulary for the given context.
- Suggested Band Score (Lexical Resource): [Insert Score]

## Grammatical Range and Accuracy:
- Evaluate the variety and complexity of sentence structures.
- Point out specific grammatical errors, such as incorrect verb forms or sentence construction and Suggest modified versions or corrections for the identified mistakes. [list of mistakes and rectify]
- Examine the use of punctuation and sentence formation.
- Suggested Band Score (Grammatical Range and Accuracy): [Insert Score]

## Overall Band Score:
- Provide an overall band score for the essay, considering the holistic performance across all criteria.
- Consider the synergy of the essay in meeting the task requirements cohesively.
- Suggested Overall Band Score: [Insert Score]

## Feedback and Additional Comments:
- Provide constructive feedback highlighting specific strengths and areas for improvement.
- Suggest strategies for enhancement in weaker areas.

## Input:
{}

## Evaluation:
{}"""

In [ ]:
print(type(dataset))

print(dataset)

<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['prompt', 'essay', 'evaluation', 'band'],
    num_rows: 9912
})


In [ ]:
import pandas as pd
import random

# Convert train_data and test_data to pandas DataFrame
train_df = pd.DataFrame(dataset)

# Function to shuffle the prompt and essay
def shuffle_input(prompt, essay):
    input_list = [prompt, essay]
    random.shuffle(input_list)  # Shuffle the prompt and essay
    return "\n\n".join(input_list)  # Join the shuffled components

# Apply the shuffle function to each row in train_df and test_df
train_df['input'] = train_df.apply(lambda row: shuffle_input(row['prompt'], row['essay']), axis=1)

dataset = train_df

print(dataset)

                                                 prompt  \
0     Young people who commit crimes should be treat...   
1     Young people who commit crimes should be treat...   
2     Young people who commit crimes should be treat...   
3     Young people who commit crimes should be treat...   
4     Young people who commit crimes should be treat...   
...                                                 ...   
9907  The best way to teach children to cooperate is...   
9908  The best way to teach children to cooperate is...   
9909  The best way to teach children to cooperate is...   
9910  In today's world, people spend a lot of money ...   
9911  Some people think the main purpose of schools ...   

                                                  essay  \
0     Deciding to choose among the potential ways of...   
1     In this modern era, youngsters who commit offe...   
2     In this modern era, youngsters who commit offe...   
3     It is suggested that punishments for immature ...

In [ ]:
from datasets import Dataset

# Correct way to drop rows with missing values in 'essay' or 'prompt' columns
dataset = dataset.dropna(subset=['essay', 'prompt'])

print(dataset)

# Convert pandas DataFrame back to Hugging Face Dataset
dataset = Dataset.from_pandas(dataset)

# Print the dataset
print(dataset)

                                                 prompt  \
0     Young people who commit crimes should be treat...   
1     Young people who commit crimes should be treat...   
2     Young people who commit crimes should be treat...   
3     Young people who commit crimes should be treat...   
4     Young people who commit crimes should be treat...   
...                                                 ...   
9907  The best way to teach children to cooperate is...   
9908  The best way to teach children to cooperate is...   
9909  The best way to teach children to cooperate is...   
9910  In today's world, people spend a lot of money ...   
9911  Some people think the main purpose of schools ...   

                                                  essay  \
0     Deciding to choose among the potential ways of...   
1     In this modern era, youngsters who commit offe...   
2     In this modern era, youngsters who commit offe...   
3     It is suggested that punishments for immature ...

In [ ]:
EOS_TOKEN = '</s>' # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    essay_input = examples["input"]
    evaluations = examples["evaluation"]
    texts = []
    for p, ev in zip(essay_input, evaluations):
      #Must dd EOS_TOKEN, otherwise your generation will go on forever!
      text = PROMPT.format(p, ev) + EOS_TOKEN
      if isinstance(text, str):
        texts.append(text)
      else:
        print("error")
        texts.append("")
    return {"text": texts}

In [ ]:
dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/9912 [00:00<?, ? examples/s]

In [ ]:
from IPython.display import display, Markdown

def display_markdown(markdown_string):
    markdown_string = markdown_string.replace("<\s>", "")
    markdown_string = markdown_string.replace("<s>", "")
    display(Markdown(markdown_string))

In [ ]:
sample = dataset[7]['text']
display_markdown(sample)


In this task, you are required to evaluate an IELTS Writing Task 2 essay. Consider the following four criteria and provide a detailed assessment for each, along with a suggested band score:

## Task Achievement:
- Evaluate how well the candidate has addressed the given task.
- Assess the clarity and coherence of the response in presenting ideas.
- Identify if the candidate has fully covered all parts of the task and supported arguments appropriately.
- Suggested Band Score (Task Achievement): [Insert Score]

## Coherence and Cohesion:
- Assess the overall organization and structure of the essay.
- Evaluate the use of linking devices to connect ideas and paragraphs.
- Identify if there is a logical flow of information.
- Suggested Band Score (Coherence and Cohesion): [Insert Score]

## Lexical Resource (Vocabulary):
- Examine the range and accuracy of vocabulary used in the essay.
- Point out specific mistakes in vocabulary, such as inaccuracies or overuse of certain words and Suggest modified versions or alternatives for the identified mistakes. [list of mistakes and rectify]
- Assess the appropriateness of vocabulary for the given context.
- Suggested Band Score (Lexical Resource): [Insert Score]

## Grammatical Range and Accuracy:
- Evaluate the variety and complexity of sentence structures.
- Point out specific grammatical errors, such as incorrect verb forms or sentence construction and Suggest modified versions or corrections for the identified mistakes. [list of mistakes and rectify]
- Examine the use of punctuation and sentence formation.
- Suggested Band Score (Grammatical Range and Accuracy): [Insert Score]

## Overall Band Score:
- Provide an overall band score for the essay, considering the holistic performance across all criteria.
- Consider the synergy of the essay in meeting the task requirements cohesively.
- Suggested Overall Band Score: [Insert Score]

## Feedback and Additional Comments:
- Provide constructive feedback highlighting specific strengths and areas for improvement.
- Suggest strategies for enhancement in weaker areas.

## Input:
Young people who commit crimes should be treated the same way as adults. To what extent do you agree or disagree?

It is argued that juvenile offenders should be given punishment as well a grown people . In my opinion, I completely do not agree with that. 

First of all, anybody deserves a second chance too and especially teenagers who are not mature enough and lack life experience. They have to learn a lot of lessons about life through society, law and others. Moreover, I suppose that young offenders might copy the behaviour of adults. Therefore, I think we should have more useful punishment for them. For instance, we can force them to work in the community unpaid or give them to a juvenile education centre to teach them more life skills in public. 

On the other hand, however, it is true that juvenile offenders who are really dangerous should receive heavy punishment and should be separated from nation. Because other young people might be impacted by them. For example, Young mudders who should be completely eliminated from civilization. Furthermore, those who are still given clemency of law may be re-offend and this is dangerous for people and society.

In conclusion, I believe that young convicts should not receive a penalty as adults do.

## Evaluation:
**Task Achievement:**

The candidate has partially addressed the given task. While they have expressed their disagreement with the statement, their response lacks a clear and coherent stance. The arguments presented are somewhat disjointed and do not fully cover all aspects of the task.

- Clarity and coherence: The essay lacks a clear structure and organization. The paragraphs do not flow logically, and the ideas are not presented in a cohesive manner.
- Covering all parts of the task: The essay briefly mentions that dangerous juvenile offenders should receive heavy punishment, but it does not provide a detailed discussion of the extent to which they should be treated like adults.

**Suggested Band Score (Task Achievement):** 5

**Coherence and Cohesion:**

The essay lacks overall organization and structure. There is no clear introduction or conclusion, and the paragraphs are not properly connected. Linking devices are not used effectively, resulting in a disjointed flow of information.

- Organization and structure: The essay lacks a logical progression of ideas. The paragraphs jump from one topic to another without proper transitions.
- Linking devices: The writer does not use linking devices consistently or appropriately. As a result, the connections between ideas are unclear.

**Suggested Band Score (Coherence and Cohesion):** 4

**Lexical Resource (Vocabulary):**

The essay demonstrates a limited range of vocabulary. The writer often repeats the same words and phrases, such as "young offenders" and "punishment." There are also several grammatical errors throughout the essay.

- Range and accuracy: The vocabulary used is basic and repetitive. The writer does not use a variety of words or expressions to convey their ideas effectively.
- Grammatical errors: There are several grammatical errors, such as "I completely do not agree with that" and "I suppose that..." These errors hinder the clarity and coherence of the essay.

**Suggested Band Score (Lexical Resource):** 5

**Grammatical Range and Accuracy:**

The essay exhibits errors in sentence structure and grammar. The sentences are often short and simple, and there is a lack of variety in sentence structure. The writer also makes several grammatical mistakes, such as incorrect verb forms and sentence construction.

- Sentence structure: The sentences are predominantly simple and lack complexity. The writer does not use a range of sentence structures to enhance the flow and readability of the essay.
- Grammatical errors: There are several grammatical errors, such as "anybody deserves a second chance too" and "I think we should have more useful punishment for them." These errors affect the overall clarity and quality of the essay.

**Suggested Band Score (Grammatical Range and Accuracy):** 5

**Overall Band Score:** 5

The essay demonstrates a limited understanding of the task and lacks overall coherence and cohesion. The writer's arguments are not fully developed or supported, and the language used is basic and repetitive. There are also several grammatical errors throughout the essay.

**Feedback and Additional Comments:**

**Strengths:**

- The writer expresses a clear opinion on the topic.

**Areas for Improvement:**

- Improve the clarity and coherence of the response by organizing ideas logically and using appropriate linking devices.
- Expand the range of vocabulary used and avoid repetition.
- Correct grammatical errors and improve sentence structure.
- Develop arguments more fully and provide specific examples to support them.

**Suggested Strategies for Enhancement:**

- Practice outlining essays to improve organization and structure.
- Use a thesaurus to expand vocabulary and find more precise and varied words.
- Review grammar rules and practice writing complex sentences.
- Read model essays and analyze their structure, language, and argumentation.</s>

#Train Model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 40, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
        hub_model_id="R0b000/Fine_tuned_Mistral_IELTS",
        push_to_hub=False,
        # prompt halnee
    ),
)

Map (num_proc=2):   0%|          | 0/9912 [00:00<?, ? examples/s]

#Show current memeory stats

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
4.031 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,912 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 40
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.253200
2,1.250400
3,1.238100
4,1.183500
5,1.131000
6,0.951600
7,0.979900
8,0.845600
9,0.749400
10,0.836400


#Show final memory and time stats

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1389.3458 seconds used for training.
23.16 minutes used for training.
Peak reserved memory = 5.979 GB.
Peak reserved memory for training = 1.948 GB.
Peak reserved memory % of max memory = 40.56 %.
Peak reserved memory for training % of max memory = 13.215 %.


In [ ]:
print(trainer_stats)

TrainOutput(global_step=40, training_loss=0.8529706880450248, metrics={'train_runtime': 1389.3458, 'train_samples_per_second': 0.23, 'train_steps_per_second': 0.029, 'total_flos': 2.285239921110221e+16, 'train_loss': 0.8529706880450248, 'epoch': 0.03228410008071025})


#Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
input = """
Young people who commit crimes should be treated in the same way as adults who commit crimes. To what extent do you agree or disagree?

Deciding to choose among the potential ways of punishing young people who commit crimes continues to be a controversial issue for the societies and the governments. It is argued by some that these people should be treated the same as adults. I personally disagree with this opinion due to the adverse effects of imprisonment on a teenager’s mental health. Many countries put the criminals of their society at jails considering it as a very effective way of punishment. It is understandable that this does exert a positive influence on decreasing crime in the society by putting the criminals in an unpleasant situation which they would mostly never wish to experience again. As a result, this could impede them from attempting crime in the future. For example, my friend who had been sent to jail for 2 months because of repeatedly committing traffic offends, has never committed the same crime since being released admitting that being in prison had been intolerable for her. However, I believe using the same way of punishment for youngsters would not be a wise idea. In fact, teenagers are at a very critical age in which the core of their personality is being shaped. There for, sending them to prison like adults as a way of punishment for their crimes, which are most often pity crimes, would actually expose them to other criminals who might have some serious personality disorders and this would adversely affect their personality as an adult in the future. To put in another way, such punishments are potential to become a threat to their mental health leading them to commit more serious crimes in a long run. For instance, according to the law of my country, young people are being punished the same as adults. A recent survey revealed that this policy has not been effective so far since 60% of these teenage criminals end up drug trafficking after being released from prison which had been sent to for a pity crime. In conclusion, although the ways that adults are being punished in many countries might be quite effective to decrease crime rate, I do not agree that it is a wise decision to use these ways for punishing young people as well.

"""

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    PROMPT.format(
        input,
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True, temperature=0.8, do_sample=True)
answer = tokenizer.batch_decode(outputs)
evaluation = answer[0].split("Evaluation:")[1]

display_markdown(evaluation)


## Task Achievement:
- The candidate has partially addressed the task by considering the adverse effects of imprisonment on a teenager's mental health.
- The essay is not very clear or coherent in presenting ideas, and the candidate has not fully covered all parts of the task.
- The arguments are not well-supported or developed, and the candidate has not considered alternative perspectives.
- Suggested Band Score (Task Achievement): 4.5

## Coherence and Cohesion:
- The overall organization and structure of the essay are weak.
- There is a lack of logical flow of information, and the paragraphs do not connect well.
- The use of linking devices is limited, and the essay lacks cohesion.
- Suggested Band Score (Coherence and Cohesion): 4.0

## Lexical Resource (Vocabulary):
- The candidate has used a limited range of vocabulary.
- There are several errors in vocabulary, such as "adverse effects" and "intolerable" should be "negative impacts" and "unpleasant."
- The vocabulary is often inaccurate or overused.
- Suggested Band Score (Lexical Resource): 4.5

## Grammatical Range and Accuracy:
- The candidate has used a limited range of sentence structures.
- There are several grammatical errors, such as "admitting that being in prison had been intolerable for her" should be "admitting that being in prison had been unpleasant for her."
- The use of punctuation and sentence formation is often incorrect.
- Suggested Band Score (Grammatical Range and Accuracy): 4.0

## Overall Band Score:
- The overall band score for the essay is 4.25.
- The essay fails to meet the requirements of the task and lacks coherence, cohesion, and grammatical accuracy.
- The candidate demonstrates a limited understanding of the topic and has not fully explored the potential adverse effects of imprisonment on a teenager's mental health.

## Feedback and Additional Comments:
- The candidate needs to improve their ability to organize and structure their ideas effectively.
- The candidate should also work on expanding their vocabulary and improving their grammatical accuracy.
- The candidate could benefit from reading widely and practicing writing essays to improve their overall writing skills.
- The candidate could also research more about the topic and gather more evidence to support their arguments.</s>

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    PROMPT.format(
        input,
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048, temperature=0.777, do_sample=False)

<s> 
In this task, you are required to evaluate an IELTS Writing Task 2 essay. Consider the following four criteria and provide a detailed assessment for each, along with a suggested band score:

## Task Achievement:
- Evaluate how well the candidate has addressed the given task.
- Assess the clarity and coherence of the response in presenting ideas.
- Identify if the candidate has fully covered all parts of the task and supported arguments appropriately.
- Suggested Band Score (Task Achievement): [Insert Score]

## Coherence and Cohesion:
- Assess the overall organization and structure of the essay.
- Evaluate the use of linking devices to connect ideas and paragraphs.
- Identify if there is a logical flow of information.
- Suggested Band Score (Coherence and Cohesion): [Insert Score]

## Lexical Resource (Vocabulary):
- Examine the range and accuracy of vocabulary used in the essay.
- Point out specific mistakes in vocabulary, such as inaccuracies or overuse of certain words and Sugg

In [ ]:
import json
from transformers import AutoConfig

# Load the config file
config = AutoConfig.from_pretrained("R0b000/Fine_tuned_Mistral_IELTS")

# Add the pipeline tag
config.pipeline_tag = "text-generation"  # Change to your task, e.g., "text-classification"

# Save the updated config
config.save_pretrained("R0b000/Fine_tuned_Mistral_IELTS")


config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

#Saving Model

In [ ]:
trainer.push_to_hub()

  0%|          | 0/2 [00:00<?, ?it/s]